In [ ]:
%matplotlib notebook

In [ ]:
import hyperspy.api as hs
from stemutils.io import *
from stemutils.visualise import *

In [ ]:
import h5py as h5
def get_stage_pos_from_hdf(fp):
    md = h5.File(fp, 'r')
    return np.array((md['metadata']['x_pos(m)'][()], md['metadata']['y_pos(m)'][()]))

def get_img_stage_coords(img):
    x = img.original_metadata['ImageList']['TagGroup0']['ImageTags']['Microscope Info']['Stage Position']['Stage X']
    y = img.original_metadata['ImageList']['TagGroup0']['ImageTags']['Microscope Info']['Stage Position']['Stage Y']
    return [x,y]

def prep_coord_array(arr):
    return np.pad(arr, ((0,0),(0,1)), mode = 'constant', constant_values = 0.0)
def regress_coord_affine_transform(primary, secondary):  
    primary = prep_coord_array(primary)
    secondary = prep_coord_array(secondary)
    n = primary.shape[0]
    pad = lambda x: np.hstack([x, np.ones((x.shape[0], 1))])
    unpad = lambda x: x[:,:-1]
    X = pad(primary)
    Y = pad(secondary)

    # Solve the least squares problem X * A = Y
    # to find our transformation matrix A
    A, res, rank, s = np.linalg.lstsq(X, Y)

    transform = lambda x: unpad(np.dot(pad(x), A))
    
    return transform, (A,res,rank)

In [ ]:
root_dir = Path('/dls/e02/data/2022/nt32759-1/raw/low_MAG_P2/')

In [ ]:
root_dir.walk('dm3')

## OneView Calibration

First thing I am doing is using some higher mag images, with particles I'll be able to find in the low mag image to calibrate a coordinate transform (this transform will convert the coordinates in the metadata into coordinates within the low mag image)

In [ ]:
low_mag_img_paths = root_dir.walk('34.dm3')

In [ ]:
low_mag_img_paths

In [ ]:
low_mag_img = hs.load(low_mag_img_paths[0])

In [ ]:
low_mag_img.plot()

In [ ]:
low_mag_img1 = hs.load(root_dir.walk('44.dm3')[0])

low_mag_img1.plot()

In [ ]:
low_mag_img2 = hs.load(root_dir.walk('40.dm3')[0])

In [ ]:
low_mag_img2.plot()

In [ ]:
low_mag_img3 = hs.load(root_dir.walk('37.dm3')[0])

low_mag_img3.plot()

In [ ]:
low_mag_img4 = hs.load(root_dir.walk('52.dm3')[0])

low_mag_img4.plot()

In [ ]:
low_mag_img5 = hs.load(root_dir.walk('60.dm3')[0])

low_mag_img5.plot()

In [ ]:
low_mag_img6 = hs.load(root_dir.walk('31.dm3')[0])

low_mag_img6.plot()

Here we pull the stage coordinates out of the dm3 metadata for the images of interest

In [ ]:
img_find_coord = get_img_stage_coords(low_mag_img)
img1_coord = get_img_stage_coords(low_mag_img1)
img2_coord = get_img_stage_coords(low_mag_img2)
img3_coord = get_img_stage_coords(low_mag_img3)
img4_coord = get_img_stage_coords(low_mag_img4)
img5_coord = get_img_stage_coords(low_mag_img5)
img6_coord = get_img_stage_coords(low_mag_img6)

... and for all the images

In [ ]:
imgs = []
stage_coords = []


for im_path in root_dir.walk('.dm3'):
    img = hs.load(im_path)
    imgs.append(img)
    stage_coords.append(get_img_stage_coords(img))

We can then view the distribution of stage coordinates 

In [ ]:
plt.figure()
for coords in stage_coords:
    plt.scatter(coords[0], coords[1], c='green')
    

plt.scatter(img_find_coord[0], img_find_coord[1], c='red')
plt.scatter(img1_coord[0], img1_coord[1], c='blue')
plt.scatter(img2_coord[0], img2_coord[1], c='navy')
plt.scatter(img3_coord[0], img3_coord[1], c='orange')
plt.scatter(img4_coord[0], img4_coord[1], c='purple')
plt.scatter(img5_coord[0], img5_coord[1], c='pink')
plt.scatter(img6_coord[0], img6_coord[1], c='yellow')

Now the manual part, need to find the location of the dm3 images in the low mag image and save these coordinates

In [ ]:
stage_img_path = Path('/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/I08LowMag/I08LowMag/P2_highermag/fullstage.jpg')

In [ ]:
import cv2

In [ ]:
stage_img = cv2.imread(str(stage_img_path))

In [ ]:
plt.figure(figsize = (10,10))
plt.imshow(stage_img)

In [ ]:
full_fov_img_find = [2564.,2055.8]
full_fov_img1 = [2592.3,2025.6]
full_fov_img2 = [2565.6,2160.6]
full_fov_img3 = [2555.9,2098.3]
full_fov_img4 = [2682.7,2051.8]
full_fov_img5 = [2532.3,2046.0]
full_fov_img6 = [2839.6,1063.6]

In [ ]:
plt.figure(figsize = (10,10))
plt.imshow(stage_img)
plt.scatter(full_fov_img_find[0], full_fov_img_find[1], c='red')
plt.scatter(full_fov_img1[0], full_fov_img1[1], c='blue')
plt.scatter(full_fov_img2[0], full_fov_img2[1], c='navy')
plt.scatter(full_fov_img3[0], full_fov_img3[1], c='orange')
plt.scatter(full_fov_img4[0], full_fov_img4[1], c='purple')
plt.scatter(full_fov_img5[0], full_fov_img5[1], c='pink')
plt.scatter(full_fov_img6[0], full_fov_img6[1], c='yellow')

We get arrays of the stage space and image space coordinates, taking care to ensure the sets of coordinates are in the same order for both

In [ ]:
stage_space_coords = np.asarray([img_find_coord,img1_coord, img2_coord, img3_coord, img4_coord, img5_coord, img6_coord])
img_space_coords = np.asarray([full_fov_img_find,full_fov_img1, full_fov_img2, full_fov_img3, full_fov_img4, full_fov_img5, full_fov_img6])

Use `regress_coord_affine_transform` to use least squares regression to fit an affine transform between the two coordinate systems

In [ ]:
transform, _ = regress_coord_affine_transform(stage_space_coords, img_space_coords)

need to use `prep_coord_array` before using `transform` but then will get all the OneView acquisition coords in image space

In [ ]:
all_stage_coords  = prep_coord_array(np.asarray(stage_coords))
all_img_coords = transform(all_stage_coords)

In [ ]:
plt.figure(figsize = (10,10))
plt.imshow(stage_img)

plt.scatter(full_fov_img_find[0], full_fov_img_find[1], c='red')
plt.scatter(full_fov_img1[0], full_fov_img1[1], c='blue')
plt.scatter(full_fov_img2[0], full_fov_img2[1], c='navy')
plt.scatter(full_fov_img3[0], full_fov_img3[1], c='orange')
plt.scatter(full_fov_img4[0], full_fov_img4[1], c='purple')

for ind, trans_im_coord in enumerate(all_img_coords):
    oneview_num = root_dir.walk('.dm3')[ind].parts[-1].split('.')[0].split('_')[-1]
    
    plt.scatter(trans_im_coord[0], trans_im_coord[1], c='green', alpha = 0.9)
    plt.annotate(oneview_num, (trans_im_coord[0], trans_im_coord[1]), c = 'red')

In [ ]:
one_stage_sig = hs.signals.Signal2D(np.linalg.norm(stage_img, axis = -1))

Finally use hyperspy permanent markers to save the locations into a hdf5 file 

In [ ]:
for ind, trans_im_coord in enumerate(all_img_coords):
    oneview_num = root_dir.walk('.dm3')[ind].parts[-1].split('.')[0].split('_')[-1]
    
    pm = hs.plot.markers.point(trans_im_coord[0], trans_im_coord[1], color='green')
    tm = hs.plot.markers.text(trans_im_coord[0], trans_im_coord[1], oneview_num, color = 'red', ha='center')
    
    one_stage_sig.add_marker(pm, permanent=True)
    one_stage_sig.add_marker(tm, permanent=True)

In [ ]:
one_stage_sig.save('/dls/e02/data/2022/nt32759-1/processing/Merlin/AnnotatedMaps/P2_Oneviews.hdf5')

## SEND acquisition calibration

Now repeat essentially the same process for the SEND datasets, except this time it will be harder to find the correct particles so will probably have to use the OneView as an intermediate

In [ ]:
for p in root_dir.walk('dm3'):
    hs.load(p).plot()

In [ ]:
ibf_paths = Path('/dls/e02/data/2022/nt32759-1/processing/Merlin/P2').walk('ibf.jpg', max_depth=1)

In [ ]:
ibf_paths

In [ ]:
import imageio

In [ ]:
for ibfp in ibf_paths:
    plt.figure()
    plt.imshow(imageio.imread(str(ibfp)))
    plt.title(str(ibfp))

In [ ]:
hs.load(root_dir.walk('0034.dm3')[0]).plot()

In [ ]:
plt.figure(figsize = (10,10))
plt.imshow(stage_img)

plt.scatter(full_fov_img_find[0], full_fov_img_find[1], c='red')
plt.scatter(full_fov_img1[0], full_fov_img1[1], c='blue')
plt.scatter(full_fov_img2[0], full_fov_img2[1], c='navy')
plt.scatter(full_fov_img3[0], full_fov_img3[1], c='orange')
plt.scatter(full_fov_img4[0], full_fov_img4[1], c='purple')

for ind, trans_im_coord in enumerate(all_img_coords):
    oneview_num = root_dir.walk('.dm3')[ind].parts[-1].split('.')[0].split('_')[-1]
    
    plt.scatter(trans_im_coord[0], trans_im_coord[1], c='green', alpha = 0.9)
    plt.annotate(oneview_num, (trans_im_coord[0], trans_im_coord[1]), c = 'red', fontsize = 15)

134905 == oneview 0046
142950 == oneview 0034
135446 == oneview 0047

In [ ]:
particle905 = [2603.1,2058.7]
particle950 = [2566.0,2054.6]
particle446 = [2609.1,2076.1]

In [ ]:
md_path = Path('/dls/e02/data/2022/nt32759-1/processing/Merlin/P2').walk('.hdf', 'hdf5')

In [ ]:
md_coords = [get_stage_pos_from_hdf(x) for x in md_path]

In [ ]:
found_md_paths = [x for x in md_path if str(x).find('905') != -1 or str(x).find('950') != -1 or str(x).find('446') != -1 ]

In [ ]:
found_md_paths

In [ ]:
reference_stage_coords = np.asarray((particle905, particle446, particle950))

In [ ]:
plt.figure(figsize = (10,10))
plt.imshow(stage_img)


plt.scatter(reference_stage_coords[0,0], reference_stage_coords[0,1], c='blue')
plt.scatter(reference_stage_coords[1,0], reference_stage_coords[1,1], c='green')
plt.scatter(reference_stage_coords[2,0], reference_stage_coords[2,1], c='orange')
#plt.scatter(reference_stage_coords[3,0], reference_stage_coords[3,1], c='red')
#plt.scatter(reference_stage_coords[4,0], reference_stage_coords[4,1], c='purple')
#plt.scatter(reference_stage_coords[5,0], reference_stage_coords[5,1], c='pink')
#plt.scatter(reference_stage_coords[6,0], reference_stage_coords[6,1], c='brown')

In [ ]:
found_particle_coords = np.asarray([get_stage_pos_from_hdf(x) for x in found_md_paths])

In [ ]:
diff_trans, _ = regress_coord_affine_transform(found_particle_coords, reference_stage_coords)

In [ ]:
diff_sample_locations = diff_trans(prep_coord_array(md_coords))

In [ ]:
plt.figure(figsize = (10,10))
plt.imshow(stage_img)

for i, p in enumerate(md_path):
    plt.scatter(diff_sample_locations[i,0], diff_sample_locations[i,1])
    plt.annotate(p.parts[-1], (diff_sample_locations[i,0], diff_sample_locations[i,1]), c = 'red')

In [ ]:
add_annotation_markers('/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/I08LowMag/I08LowMag/P2Sample3/samplegridannotated.dm3')

In [ ]:
stage_sig = hs.signals.Signal2D(np.linalg.norm(stage_img, axis = -1))

In [ ]:
for i, p in enumerate(md_path):
    pm = hs.plot.markers.point(diff_sample_locations[i,0], diff_sample_locations[i,1], color = 'red')
    tm = hs.plot.markers.text(diff_sample_locations[i,0], diff_sample_locations[i,1],p.parts[-1], color = 'red', ha='center')
    stage_sig.add_marker(pm,permanent=True)
    stage_sig.add_marker(tm, permanent= True)


In [ ]:
'/dls/e02/data/2022/nt32759-1/processing/Merlin/AnnotatedMaps/P2_samples.hdf5'

In [ ]:
stage_sig.save('/dls/e02/data/2022/nt32759-1/processing/Merlin/AnnotatedMaps/P2_samples.hdf5')